## Load libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import os
import sys
sys.path.append("../libs")
sys.path.append("../")
from definitions import ROOT_DIR
import utils as utils

# Set pandas options
# Not use scientific notation and use two decimal places and use comma as thousand separator
pd.options.display.float_format = '{:,.2f}'.format


# ETL

## 1. Load dataset

In [2]:
# Load dataset from data_files/internet.xlsx - sheet: 'Ingresos '
df = utils.get_xls_sheet_data('/data_files/internet.xlsx', 'Velocidad % por prov')
df.head().style.format(thousands=',')

,Año,Trimestre,Provincia,Mbps (Media de bajada)
0,"2,024",2,Buenos Aires,157.410000
1,"2,024",2,Capital Federal,233.010000
2,"2,024",2,Catamarca,97.380000
3,"2,024",2,Chaco,107.760000
4,"2,024",2,Chubut,21.670000


## 2. Handling missing values

### Look for missing values in all cells

In [3]:
#Find missing values
df.isnull().sum()

Año                       0
Trimestre                 0
Provincia                 0
Mbps (Media de bajada)    0
dtype: int64

#### There's no missing values

## 3. Look for Duplicates

### Find duplicates for complete rows

In [4]:
#Find duplicates by complete row
df.duplicated().sum()

0

#### There's no complete duplicated rows

### Fin duplicated rows for year and quarter ('Trimestre') and state ('Provincia')

In [5]:
#Find duplicated rows by row, for year and quarter
df.duplicated(subset=['Año', 'Trimestre', 'Provincia']).sum()

0

#### There's no duplicated rows for year and quarter

## 4. Finding outliers

In [6]:
#Finding outliers
df.describe()

,Año,Trimestre,Mbps (Media de bajada)
count,"1,008.00","1,008.00","1,008.00"
mean,"2,018.76",2.45,24.72
std,3.04,1.12,33.66
min,"2,014.00",1.00,2.57
25%,"2,016.00",1.00,4.87
50%,"2,019.00",2.00,10.09
75%,"2,021.00",3.00,29.88
max,"2,024.00",4.00,234.44


#### In the statistics, we can see there's no outliers values for the column year ("Año"), because the maximum value is 2024 and the data is historical. For the quarter ("Trimestre") column the minimum an maximum values are correct, 1 and 4. For the MBPS column the distance from max value are far from mean plus std, but the max value are in the same proportion of income. No outlies detected.

In [7]:
df.groupby(['Año','Trimestre'])['Provincia'].agg(['count'])

count
Año  Trimestre       
2014 1             24
     2             24
     3             24
     4             24
2015 1             24
     2             24
     3             24
     4             24
2016 1             24
     2             24
     3             24
     4             24
2017 1             24
     2             24
     3             24
     4             24
2018 1             24
     2             24
     3             24
     4             24
2019 1             24
     2             24
     3             24
     4             24
2020 1             24
     2             24
     3             24
     4             24
2021 1             24
     2             24
     3             24
     4             24
2022 1             24
     2             24
     3             24
     4             24
2023 1             24
     2             24
     3             24
     4             24
2024 1             24
     2             24

#### All pairs year-quarter are present, there's no missing data for each of the 24 provinces.

## 5. Data Types

### We will review the data types for each column.

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1008 entries, 0 to 1007
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Año                     1008 non-null   int64  
 1   Trimestre               1008 non-null   int64  
 2   Provincia               1008 non-null   object 
 3   Mbps (Media de bajada)  1008 non-null   float64
dtypes: float64(1), int64(2), object(1)
memory usage: 31.6+ KB


#### The 'Provincia' column have an object data type, and have strings values. We should change the data type to string.

In [9]:
#Change provincia to string
df['Provincia'] = df['Provincia'].astype('string')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1008 entries, 0 to 1007
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Año                     1008 non-null   int64  
 1   Trimestre               1008 non-null   int64  
 2   Provincia               1008 non-null   string 
 3   Mbps (Media de bajada)  1008 non-null   float64
dtypes: float64(1), int64(2), string(1)
memory usage: 31.6 KB


## New columns

### Create a new column with the year and quarter

In [10]:
#Create a new column with the quarter and year
df['Periodo'] = (df['Año'].astype(str) + 'T' + df['Trimestre'].astype(str)).astype('string')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1008 entries, 0 to 1007
Data columns (total 5 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Año                     1008 non-null   int64  
 1   Trimestre               1008 non-null   int64  
 2   Provincia               1008 non-null   string 
 3   Mbps (Media de bajada)  1008 non-null   float64
 4   Periodo                 1008 non-null   string 
dtypes: float64(1), int64(2), string(2)
memory usage: 39.5 KB


# Save dataset

In [11]:
df=df.reset_index()
df.to_parquet(ROOT_DIR + '/data_files/velocidad_por_provincia_clean.parquet')